In [2]:
import numpy as np

In [3]:

with open('crafting_times.csv','r') as infile:
    header = infile.readline().rstrip().split(',')
    data = []
    for line in infile:
        data.append([float(s) for s in line.rstrip().split(',')])
    data = np.array(data)

In [27]:

validation_split = 0.3
Y = data[int(data.shape[0]*validation_split):,0]
Y_validation = data[:int(data.shape[0]*validation_split),0]
X = data[int(data.shape[0]*validation_split):,1:]
X_validation = data[:int(data.shape[0]*validation_split),1:]

Now we will use Least Squares Regression to estimate the time cost associated with a given state and end state.  

The least squares regression coefficients are found via the equation:

$\beta =  (X^T X)^{-1} X^T Y$

First try it out with using `np.dot` (anywhere there is a matrix multiplication) and `np.inv` (anywhere there is a matrix inversion. (as a note, matrix transposition is accomplished with `.T`)

Next, compare using `np.linalg.lstsq` -- numpy's built in least squares regression (that is much more stable than using the matrix inversion found here).

As another note, you might want to try adding a constant factor to your independent variables (done via `np.hstack((np.ones(# of data points, 1), X)` )

In [ ]:
#Least Squares Estimation Goes Here

B = #Your code here

Now we want to test our coefficients and see how well we predict the answer.

In [ ]:
Yhat = np.dot(X,B)


residual  = Y[:]-Yhat[:,0]

plt.plot(Y,residual,'x')
Yhat_validation = np.dot(X_validation,B)


residual_validation  = Y_validation[:]-Yhat_validation[:,0]
plt.plot(Y_validation,residual_validation,'ro')
plt.show()


print(np.mean(np.sqrt(residual*residual)))
print(np.mean(np.sqrt(residual_validation*residual_validation)))



Now we are going to use artificial neural networks.  First let's use stochastic gradient descent to do the calculations we did above.  

To do this, add a `Dense` layer in the list passed to `Sequential` -- the parameters you care about are:

`Dense(units,input_shape)`

`units` is the dimensionality of the output space -- in this case, it will be 1 (all of our final `units` will always be 1, as our output is the single number we are predicting).  `input_shape` is always required on the first layer, as Keras doesn't know how big our input is (unless we tell it).  After the first layer (not applicable here), we don't need to specify `input_shape` as it can be inferred from the output of the previous layer.  In this case, you will want to set it to `(X.shape[1],))`, i.e. a tuple (of size 1) with a single number (the number of columns found in our X data)

The summary of this network should look like it does below.

Keras requires us to compile our model before we fit it.  This compilation requires two parameters, what optimization technique we are using and what our loss function is.  Here, we are using RMS Prop, a common optimization technique, and our loss is `mse` (Mean Square Error), which is the exact same loss that the least squares regression uses.

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    #Add a single dense layer
])


model.compile(optimizer='rmsprop',
              loss='mse')

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 1)                 35        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________
None


Given the construction and compilation, we can now fit the model.  We could add other options here, but we are going to run for 200 epochs (i.e. 200 passes through our data) with a small batch size (4) -- typically we would want a larger batch size (to take advantage of a lovely GPU), but given the small size of the model and data, we are going to keep it low here.

In [ ]:
model.fit(X, Y, epochs=200, batch_size=4)

Now we want to see how it did.  We will plot the residuals (i.e. the error) for both our training set and our validation set.  It is always important to have a validation set, as it will let us see how well our model is over (or under) fitting the data.

In [ ]:
Yhat = model.predict(X)


residual  = Y[:]-Yhat[:,0]

plt.plot(Y,residual,'x')
Yhat_validation = model.predict(X_validation)


residual_validation  = Y_validation[:]-Yhat_validation[:,0]
plt.plot(Y_validation,residual_validation,'ro')
plt.show()


print(np.mean(np.sqrt(residual*residual)))
print(np.mean(np.sqrt(residual_validation*residual_validation)))


Now let's try it with some hidden layers.  Instead of an array with just 1 Dense layer, we will have multiple layers.  As with the last one, we will have to specify the input_shape (identical to that one).  However, instead of going to a vector of size 1, let's have it go to a vector of size 100.  We will have another hidden layer, also of size 100, which will finally go to another densely connected layer of size 1.

Again, your model summary should look similar to below (layer names will be different)


In [40]:


model = Sequential([
    
])
model.compile(optimizer='rmsprop',
              loss='mse')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               3500      
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 101       
Total params: 13,701
Trainable params: 13,701
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X, Y, epochs=200, batch_size=4)

In [ ]:
Yhat = model.predict(X)


residual  = Y[:]-Yhat[:,0]

plt.plot(Y,residual,'x')
Yhat_validation = model.predict(X_validation)


residual_validation  = Y_validation[:]-Yhat_validation[:,0]
plt.plot(Y_validation,residual_validation,'ro')
plt.show()


print(np.mean(np.sqrt(residual*residual)))
print(np.mean(np.sqrt(residual_validation*residual_validation)))

Wait, that looks just like it did before!  The key to neural networks comes from the non-linear activations.  No matter how many layers we add, so long as the rank of the hidden layers is $\geq$ the rank of the original vector, the best we can do is the least squares estimation (as it is the maximum likelihood estimator for a linear regression).  If the rank is decreased, then we are doing some form of compression, akin to Principal Component Analysis.  Let's try it with a bit of nonlinearity.  Let's do a single hidden layer with a non-linear activation -- we will use the Rectified Linear Unit (relu) as it is fast and all we really care about is ANY kind of nonlinearity (sometimes we care about our nonlinearity having a specific meaning or mapping into a specific range (0 to 1, -1 to 1, etc.)

In [41]:


model = Sequential([
    
])
model.compile(optimizer='rmsprop',
              loss='mse')

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 34)                1190      
_________________________________________________________________
activation_6 (Activation)    (None, 34)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 35        
Total params: 1,225
Trainable params: 1,225
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X, Y, epochs=200, batch_size=4)

In [ ]:
Yhat = model.predict(X)


residual  = Y[:]-Yhat[:,0]

plt.plot(Y,residual,'x')
Yhat_validation = model.predict(X_validation)


residual_validation  = Y_validation[:]-Yhat_validation[:,0]
plt.plot(Y_validation,residual_validation,'ro')
plt.show()


print(np.mean(np.sqrt(residual*residual)))
print(np.mean(np.sqrt(residual_validation*residual_validation)))

Ah, that looks better.  Note, you are probably overfitting (it is the tendency of these techniques), you can try some form of regularization (e.g., dropout) to reduce this overfitting for extra-credit.  If you really wanted to go extra, you could use this as the heuristic for our earlier searches and see how that speeds up the process.